# Prototype for Twitter Sentiment Analysis for Zoom Stock

This Prototype will implement the skeleton of the final project.

From Step 7, this step will use SVM and RoBERTA (non-fine tuned) to analyze 5000 zoom stock tweets.


using labeled tweets to train SVM and sentiment analysis  of zoom tweets


In [1]:
# load tweets. note that final project will have all tweets that have sentiment analysis done.

import pandas as pd




# zm_tweet_df = pd.read_csv('zm-tweet-2020-2-26-to-2020-2-29.csv')
zm_tweet_df = pd.read_csv('zm-tweets-jan to april 2020.csv')


# print(zm_tweet_df.head())

print(zm_tweet_df[['date', 'cashtags']].head())

# print(zm_tweet_df['date'].tail())
print(zm_tweet_df[['date', 'cashtags']].tail())

print(zm_tweet_df.shape)

                        date  \
0  2020-04-14 23:59:44+00:00   
1  2020-04-14 23:54:15+00:00   
2  2020-04-14 23:38:17+00:00   
3  2020-04-14 23:27:05+00:00   
4  2020-04-14 23:23:10+00:00   

                                            cashtags  
0                                             ['ZM']  
1                                             ['ZM']  
2                                             ['ZM']  
3                                             ['ZM']  
4  ['SPX', 'SPX', 'TSLA', 'NVDA', 'ZM', 'AMZN', '...  
                           date  \
5679  2020-04-01 00:28:56+00:00   
5680  2020-04-01 00:23:37+00:00   
5681  2020-04-01 00:19:37+00:00   
5682  2020-04-01 00:19:18+00:00   
5683  2020-04-01 00:03:24+00:00   

                                            cashtags  
5679                                  ['GOOG', 'ZM']  
5680  ['TSLA', 'BYND', 'ZM', 'DOCU', 'NFLX', 'TDOC']  
5681                                  ['ZM', 'TSLA']  
5682           ['ZM', 'MSFT', 'WORK', 'RNG', '

# Training SVM

In [2]:
# loading labelled sentiment twitter posts


# import pandas as pd
# import re


import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import time
import re

labeled_tweet_df = pd.read_csv("dataset-labled-tweets-original.csv")

In [3]:
# cleaning tweets for sentiment analysis
import re

def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)|😉', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)

    return tweet

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002500-\U00002BEF"  # chinese char
                           u"\U00002702-\U000027B0"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U0001f926-\U0001f937"
                           u"\U00010000-\U0010ffff"
                           u"\u2640-\u2642"
                           u"\u2600-\u2B55"
                           u"\u200d"
                           u"\u23cf"
                           u"\u23e9"
                           u"\u231a"
                           u"\ufe0f"  # dingbats
                           u"\u3030"
                           "]+", flags=re.UNICODE)
    
    return emoji_pattern.sub(r'', string)


import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

def preprocess_word(word):
    # Tokenize the word
    tokens = nltk.word_tokenize(word)
    
    # Convert to lowercase
    tokens = [token.lower() for token in tokens]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Stem the tokens
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join the tokens back into a single string
    preprocessed_word = ' '.join(tokens)
    
    return preprocessed_word



def preprocess_tweet(tweet):
    processed_tweet = []
    # Convert to lower case
    tweet = tweet.lower()
    
    #Clean only digits
    tweet = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", tweet)
    
    # Replaces URLs with the word URL
    #tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', '', tweet)
    
    # Replace @handle with the word USER_MENTION
    #tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    tweet = re.sub(r'@[\S]+', '', tweet)
    
    # Replaces #hashtag with hashtag
    #tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    tweet = re.sub(r'#(\S+)', '', tweet)
    
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')

    # Replace emojis with either EMO_POS or EMO_NEG
    #tweet = handle_emojis(tweet)
    tweet = remove_emoji(tweet)
   
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)

    #my custom chars
    tweet = tweet.replace('₺','')
    tweet = tweet.replace('=','')
    tweet = tweet.replace('’','')
    tweet = tweet.replace('|','')
    tweet = tweet.replace('‘','')
    tweet = tweet.replace('/','')
    tweet = tweet.replace('…','')
    tweet = tweet.replace('–','')
    tweet = tweet.replace('&','')
    tweet = tweet.replace('“','')
    tweet = tweet.replace('”','')
    tweet = tweet.replace('+','')
    tweet = tweet.replace('%','')
    tweet = tweet.replace('@','')
    tweet = tweet.replace('#','')

    words = word_tokenize(tweet) #tweet.split()

    for word in words:
      word = preprocess_word(word)
      #if is_valid_word(word):
      #    processed_tweet.append(word)
      processed_tweet.append(word)

    return ' '.join(processed_tweet)


from nltk import word_tokenize

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

def clean_tweet(tweet):
    tweet = str(tweet)
    handle_emojis(tweet)
    remove_emoji(tweet)
    preprocess_tweet(tweet)
    return str(tweet) 




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\jk\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
labeled_tweet_df["clean_tweet"] = labeled_tweet_df['text'].apply(clean_tweet) # applies clean_tweet function on each record of 'text' and creates a new column 'clean_tweet'

In [ ]:
labeled_tweet_df.head()

,textID,text,selected_text,sentiment,clean_tweet
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,"I`d have responded, if I were going"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,Sooo SAD I will miss you here in San Diego!!!
2,088c60f138,my boss is bullying me...,bullying me,negative,my boss is bullying me...
3,9642c003ef,what interview! leave me alone,leave me alone,negative,what interview! leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"Sons of ****, why couldn`t they put them on t..."


In [ ]:
labeled_tweet_df.shape

(27481, 5)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(labeled_tweet_df, test_size=0.2)
test.head

<bound method NDFrame.head of            textID                                               text  \
9879   81a05efb02                        I`am back to Tweetie 1.1.1.   
18790  26149e0547      Cross Country was today...Hmmphh my feet hurt   
9134   035e3b10ef        Taking my brother to the airport  Then gym.   
6649   c4aa8ca103                                Raining in Calicut.   
2669   1e8baf3c3e     had a **** contact with both parents >.>  grrr   
...           ...                                                ...   
10159  c774e791ab  Ugh.  Las #vegas airport is at a 'ground stop'...   
25194  04682e718c                            really wants a puppy...   
7945   b804b59377   that is sickening... Just shoot at will? Smh....   
12990  c280ab00dd   glad its nice there. Gray skies and rainy in ...   
22174  7d85365781  _sis He got those cars before he lost his job....   

                                           selected_text sentiment  \
9879                         I`am b

In [ ]:

vectorizer = CountVectorizer(max_features=10000)
BOW = vectorizer.fit_transform(labeled_tweet_df['clean_tweet'])


from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(BOW,np.asarray(labeled_tweet_df["sentiment"]))




from sklearn.svm import SVC
model = SVC()
model.fit(x_train,y_train)


SVC()

In [ ]:
predictions = model.predict(x_test)

In [ ]:
print(type(predictions))

print(len(predictions))

<class 'numpy.ndarray'>
6871


In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix

print("Accuracy of model is {}%".format(accuracy_score(y_test,predictions) * 100))


# note that accuracy changes each time: so far ranging from 67.37% to 69%

Accuracy of model is 67.37010624363265%


# Clean Zoom Tweets



In [ ]:
# cleaning zoom tweets using the following method
# df["clean_tweet"] = df['text'].apply(clean_tweet) # applies clean_tweet function on each record of 'text' and creates a new column 'clean_tweet'

zm_tweet_df['clean_tweet'] = zm_tweet_df['content'].apply(clean_tweet)



In [ ]:
zm_tweet_df[['clean_tweet', 'cashtags']].tail()

,clean_tweet,cashtags
5679,My kids are getting an incredible crash course...,"['GOOG', 'ZM']"
5680,Fellow #StockNerds...this one thing determines...,"['TSLA', 'BYND', 'ZM', 'DOCU', 'NFLX', 'TDOC']"
5681,$ZM is like $TSLA without the \n* IP\n* Unique...,"['ZM', 'TSLA']"
5682,$ZM Zoom's active users up 151% this month ht...,"['ZM', 'MSFT', 'WORK', 'RNG', 'DOCU']"
5683,Thought about selling my $ZM shares until I le...,['ZM']


# Using Trained SVM Model for sentiment analysis for 5k Zoom Twitter Dataset

In [ ]:
x_test

<6871x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 74798 stored elements in Compressed Sparse Row format>

In [ ]:
# run svm on 5k dataset 
    # create a new column "svm_zm_sentiment" on zm_tweet_df['clean_tweet']
        #setup SVM for sentiment analysis - setting up Bag of Words (BOW) - note that this was all done earlier
new_BOW = vectorizer.transform(zm_tweet_df['clean_tweet'])

        #create the ndarray predictions_svm_zm to be added as a new column back into zm_tweet_df
predictions_svm_zm = model.predict(new_BOW)
        #creating the new column 'svm_sentiment'
zm_tweet_df['sentiment_svm'] = predictions_svm_zm


# about 60 sec for SVM for 5k tweets note how much time passed. to compare to RoBERTa sentiment analysis





In [ ]:
# quick look at svm_sentiment

zm_tweet_df[['date', 'clean_tweet', 'sentiment_svm']].tail()

zm_tweet_df['sentiment_svm'].value_counts()

neutral     5094
positive     423
negative     167
Name: sentiment_svm, dtype: int64

In [ ]:

# Save to entire df to csv 'zm_5k_tweets_step1_with_svm.csv'
zm_tweet_df.to_csv('zm_5k_tweets_step1_with_svm.csv')



# RoBERTa - no finetuning twitter analysis on 5k twitter dataset




In [5]:
# code to load RoBERTa from step 7 (this step, no time to use CUDA cores will use old slow for loop method)
# load zm_5k_tweets_step1_with_svm.csv into zm_tweet_df (run this if not running from start of notebook)


# uncomment if running from start
zm_tweet_df = pd.read_csv('zm_5k_tweets_step1_with_svm.csv') 


In [6]:
# Prep for RoBERTa 

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import re

In [7]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

print(f"\n\nMODEL string: {MODEL}\n\n")
tokenizer = AutoTokenizer.from_pretrained(MODEL)



MODEL string: cardiffnlp/twitter-roberta-base-sentiment




In [8]:
# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [10]:
# Testing individual tweets

text = "$zm ended the day at the price of $90.95"
text = clean_tweet(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

print(f"Results on text: {text}")


ranking = np.argsort(scores)

print("ranking: " + str(ranking))
ranking = ranking[::-1]
print("ranking: " + str(ranking))
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

Results on text: $zm ended the day at the price of $90.95
ranking: [0 2 1]
ranking: [1 2 0]
1) neutral 0.896
2) positive 0.0624
3) negative 0.0417


In [11]:

MODEL_R = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model_r = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [16]:
zm_tweet_df['clean_tweet'].tail()


zm_tweet_df['sentiment_svm'].tail(30)

5654     neutral
5655    negative
5656     neutral
5657     neutral
5658     neutral
5659     neutral
5660     neutral
5661     neutral
5662     neutral
5663     neutral
5664     neutral
5665     neutral
5666     neutral
5667     neutral
5668     neutral
5669     neutral
5670     neutral
5671     neutral
5672     neutral
5673     neutral
5674    positive
5675     neutral
5676     neutral
5677     neutral
5678     neutral
5679    negative
5680     neutral
5681     neutral
5682     neutral
5683     neutral
Name: sentiment_svm, dtype: object

In [17]:
# RoBERTa predicted sentiments
predictions = []

for text in zm_tweet_df['clean_tweet']:
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model_r(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    predicted_label = config.id2label[ranking[0]]
    predictions.append(predicted_label)

In [ ]:

# print display after each 50 processed.. print row_num + seconds_passed
# when complete. note how much time passed. compare to SVM time.


In [ ]:
# prep to add new col 'r_sentiment'
df_predictions_r_series = pd.Series(predictions,  name='sentiment_r') # creating temp df series 

# create a new column in 5k zoom tweet 'r_sentiment'
zm_tweet_df = zm_tweet_df.assign(sentiment_r = df_predictions_r_series) # creating the new col


In [ ]:
# checking to see if all the sentiment cols are there
zm_tweet_df[['clean_tweet', 'sentiment_svm', 'sentiment_r']].tail(30)

In [ ]:

# save entire df with 'zm_5k_tweets_with_svm_and_r.csv'
zm_tweet_df.to_csv('zm_5k_tweets_step2_with_svn_and_roberta.csv')


# Compare differences in sentiment analysis between SVM and RoBERTa

This will help me formulate the best way to finetune RoBERTa to be more accurate.

If there is time, we should finetune using the labeled tweets too. 


In [60]:
# load df from 'zm_5k_tweets_step2_with_svn_and_roberta.csv'
zm_tweet_df = pd.read_csv('zm_5k_tweets_step2_with_svn_and_roberta.csv')

# quick check - svm -- how many positive, neutral and negative using the 1 line code
zm_tweet_df['sentiment_svm'].value_counts()
# quick check - roberta -- how many positive, neutral and negative
zm_tweet_df['sentiment_r'].value_counts()
# quick check - how many svm and roberta MATCHES
# quick check - how  many svm and roberta DOESN'T Match

# create a new column final_sentiment
    # if svm_sentiment == r_sentiment: update final_sentiment = r_sentiment 
    # at this point, there will be lots of blanks where svm_sentiment != r_sentiment

# detailed list of MISMATCH- 
    # if svm_sentiment != r_sentiment: print (clean_tweet, svm_sentiment, r_sentiment)
    # manually look at the tweet and update final_sentiment (the other matching sentiments should already filled)

# Save df to 'zm_5k_tweets_with_final_sentiment.csv'



 

# FINETUNE RoBERTa with final_sentiment. 
I assume that the svm and robeta match are already trained in the model.

Finetune using the mismatch rows with the new final_sentiment row.

Save the finetuned data 

In [ ]:
# load 'zm_5k_tweets_with_final_sentiment.csv'
# I assume that the svm and robeta match are already trained in the model.

# Finetune using the mismatch rows with the new final_sentiment row.

# Save the finetuned data 